In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("furkanakdeniz/asl-handsign-dataset-grayscaled-thresholded")

print("Path to dataset files:", path)

100%|██████████| 719M/719M [00:13<00:00, 57.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/furkanakdeniz/asl-handsign-dataset-grayscaled-thresholded/versions/1


In [2]:
import os
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
import joblib
import matplotlib.pyplot as plt
import cv2  # Pour redimensionner les images

# Chemin vers le dataset
dataset_path = r"/root/.cache/kagglehub/datasets/furkanakdeniz/asl-handsign-dataset-grayscaled-thresholded/versions/1/asl-dataset/asl-dataset"

# Alphabets disponibles dans le dataset
alphabet_array = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']

# Charger les images et leurs étiquettes
image_arr = []
image_name_value = []

# Taille cible pour toutes les images
target_size = (28, 28)

for letter in alphabet_array:
    path_letter = os.path.join(dataset_path, "train", letter)
    if not os.path.exists(path_letter):
        print(f"Le dossier {path_letter} n'existe pas.")
        continue

    for image_name in os.listdir(path_letter):
        image_path = os.path.join(path_letter, image_name)
        # Charger et redimensionner l'image
        image = plt.imread(image_path)  # Charger l'image
        if image.ndim == 3:  # Convertir en niveau de gris si l'image est RGB
            image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        resized_image = cv2.resize(image, target_size)  # Redimensionner
        image_arr.append(resized_image.flatten())  # Aplatir et ajouter à la liste
        image_name_value.append(letter)

In [3]:
# Conversion en tableaux numpy
X_data = np.array(image_arr)
y_data = np.array(image_name_value)

# Créer un mapping des étiquettes vers des entiers
label_to_int = {label: idx for idx, label in enumerate(np.unique(y_data))}
y_data = np.array([label_to_int[label] for label in y_data])

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)


In [4]:
# Paramètres pour GridSearch
param_grid = {
    'n_neighbors': [3, 5, 7],  # Nombre de voisins à considérer
    'weights': ['uniform', 'distance'],  # Poids des voisins
    'metric': ['euclidean', 'manhattan']  # Différentes métriques de distance
}

# Initialiser le modèle KNN
knn_model = KNeighborsClassifier()

# Configurer le GridSearch
grid_search = GridSearchCV(
    estimator=knn_model,
    param_grid=param_grid,
    scoring='accuracy',  # Critère pour évaluer les modèles
    cv=3,                # Validation croisée à 3 plis
    verbose=2,           # Afficher la progression
    n_jobs=-1            # Utiliser tous les cœurs disponibles
)

# Lancer la recherche
print("Lancement de la recherche d'hyperparamètres...")
grid_search.fit(X_train, y_train)

# Résultats du GridSearch
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Meilleurs hyperparamètres : {best_params}")
print(f"Meilleure précision moyenne : {best_score:.4f}")

# Évaluer le meilleur modèle sur les données de test
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision sur les données de test : {accuracy:.4f}")
print("\nRapport de classification :\n", classification_report(y_test, y_pred))

# Sauvegarder le meilleur modèle
model_filename = "knn_hand_sign_best_model.pkl"
joblib.dump(best_model, model_filename)

# Imprimer le chemin absolu du fichier créé
model_path = os.path.abspath(model_filename)
print(f"Meilleur modèle KNN sauvegardé sous : {model_path}")

Lancement de la recherche d'hyperparamètres...
Fitting 3 folds for each of 12 candidates, totalling 36 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Meilleurs hyperparamètres : {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
Meilleure précision moyenne : 0.9797
Précision sur les données de test : 0.9908

Rapport de classification :
               precision    recall  f1-score   support

           0       1.00      0.99      1.00       222
           1       0.98      1.00      0.99       248
           2       1.00      0.99      0.99       241
           3       0.98      1.00      0.99       322
           4       0.99      0.99      0.99       297
           5       0.99      0.99      0.99       306
           6       1.00      1.00      1.00       154
           7       0.99      1.00      0.99       257
           8       0.99      0.99      0.99       274
           9       0.97      1.00      0.98       296
          10       0.99      1.00      0.99       255
          11       0.98      0.99      0.98       280
          12       1.00      0.98      0.99       278
          13       0.99      0.98      0